In [9]:
import sqlite3
import os
import glob
import csv

In [10]:
# Função para tratar os valores monetários e converter para o formato DECIMAL
def tratar_valor_monetario(valor):
    return float(valor.replace('.', '').replace(',', '.'))

# Estrutura d Banco de Dados do Bolsa Família

* Tabela Saques:

AnoMesReferencia (VARCHAR): Ano e mês de referência do saque.

AnoMesCompetencia (VARCHAR): Ano e mês de competência do saque.

UF (VARCHAR): Sigla do estado do beneficiário.

CodigoMunicipioSIAFI (INTEGER): Código do município no SIAFI.

NomeMunicipioSIAFI (VARCHAR): Nome do município do beneficiário.

CPFBeneficiario (VARCHAR): CPF do beneficiário.

NISBeneficiario (VARCHAR): NIS do beneficiário.

NomeBeneficiario (VARCHAR): Nome do beneficiário.

DataSaque (DATE): Data em que foi realizado o saque.

ValorParcela (DECIMAL): Valor da parcela do benefício.



* Tabela Pagamentos:

AnoMesReferencia (VARCHAR): Ano e mês de referência do pagamento.

AnoMesCompetencia (VARCHAR): Ano e mês de competência do pagamento.

UF (VARCHAR): Sigla do estado do beneficiário.

CodigoMunicipioSIAFI (INTEGER): Código do município no SIAFI.

NomeMunicipioSIAFI (VARCHAR): Nome do município do beneficiário.

CPFBeneficiario (VARCHAR): CPF do beneficiário.

NISBeneficiario (VARCHAR): NIS do beneficiário.

NomeBeneficiario (VARCHAR): Nome do beneficiário.

ValorParcela (DECIMAL): Valor da parcela do benefício.


* Relações:

As tabelas Saques e Pagamentos não estão diretamente relacionadas entre si, mas ambas contêm informações relacionadas aos beneficiários do programa Bolsa Família. 

Os campos CPFBeneficiario, NISBeneficiario, NomeBeneficiario, e CodigoMunicipioSIAFI poderiam ser usados para unir informações entre as tabelas, se necessário.

In [3]:
# Função para criar as tabelas no banco de dados
def criar_tabelas(cursor):
    # Criar a tabela para Saques
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Saques (
        AnoMesReferencia TEXT,
        AnoMesCompetencia TEXT,
        UF TEXT,
        CodigoMunicipioSIAFI INTEGER,
        NomeMunicipioSIAFI TEXT,
        CPFBeneficiario TEXT,
        NISBeneficiario TEXT,
        NomeBeneficiario TEXT,
        DataSaque DATE,
        ValorParcela DECIMAL
    )
    ''')

    # Criar a tabela para Pagamentos
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS Pagamentos (
        AnoMesReferencia TEXT,
        AnoMesCompetencia TEXT,
        UF TEXT,
        CodigoMunicipioSIAFI INTEGER,
        NomeMunicipioSIAFI TEXT,
        CPFBeneficiario TEXT,
        NISBeneficiario TEXT,
        NomeBeneficiario TEXT,
        ValorParcela DECIMAL
    )
    ''')

In [11]:
# Função atualizada para processar cada arquivo CSV e inserir os dados nas tabelas apropriadas usando pandas
def processar_e_inserir_arquivos_pandas(pasta_raiz, cursor):
    subpastas = glob.glob(os.path.join(pasta_raiz, '2020??_BolsaFamilia_*'))

    for subpasta in subpastas:
        tipo = 'Saques' if 'Saques' in subpasta else 'Pagamentos'
        arquivos_csv = glob.glob(os.path.join(subpasta, '*.csv'))

        for arquivo_csv in arquivos_csv:
            df = pd.read_csv(arquivo_csv, delimiter=';', encoding='ISO-8859-1')

            for index, row in df.iterrows():
                if tipo == 'Saques':
                    DataSaque = f"{row['DataSaque'].split('/')[2]}-{row['DataSaque'].split('/')[1]}-{row['DataSaque'].split('/')[0]}"
                    ValorParcela = tratar_valor_monetario(row['ValorParcela'])
                    cursor.execute('''
                        INSERT INTO Saques (
                            AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI,
                            CPFBeneficiario, NISBeneficiario, NomeBeneficiario, DataSaque, ValorParcela
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (row['AnoMesCompetencia'], row['AnoMesReferencia'], row['UF'], row['CodigoMunicipioSIAFI'],
                          row['NomeMunicipioSIAFI'], row['CPFBeneficiario'], row['NISBeneficiario'],
                          row['NomeBeneficiario'], DataSaque, ValorParcela))
                else:
                    ValorParcela = tratar_valor_monetario(row['ValorParcela'])
                    cursor.execute('''
                        INSERT INTO Pagamentos (
                            AnoMesCompetencia, AnoMesReferencia, UF, CodigoMunicipioSIAFI, NomeMunicipioSIAFI,
                            CPFBeneficiario, NISBeneficiario, NomeBeneficiario, ValorParcela
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (row['AnoMesCompetencia'], row['AnoMesReferencia'], row['UF'], row['CodigoMunicipioSIAFI'],
                          row['NomeMunicipioSIAFI'], row['CPFBeneficiario'], row['NISBeneficiario'],
                          row['NomeBeneficiario'], ValorParcela))


In [13]:
# Caminho da pasta raiz onde as subpastas estarão localizadas
pasta_raiz = '/projetos/Mestrado/FBD' 

# Conectar ao banco de dados
conn = sqlite3.connect('BolsaFamilia.db')
cursor = conn.cursor()

# Criar tabelas
# criar_tabelas(cursor)


# Chamar a função para processar e inserir os arquivos
processar_e_inserir_arquivos(pasta_raiz, cursor)

# Salvar as alterações e fechar a conexão com o banco de dados
conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('BolsaFamilia.db')
cursor = conn.cursor()

# Consulta para selecionar os primeiros 5 registros da tabela Saques
cursor.execute('SELECT * FROM Saques LIMIT 10')
saques_samples = cursor.fetchall()

# Consulta para selecionar os primeiros 5 registros da tabela Pagamentos
cursor.execute('SELECT * FROM Pagamentos LIMIT 10')
pagamentos_samples = cursor.fetchall()

# Fechar a conexão após a consulta
conn.close()

# Retornar os resultados das consultas como exemplos
(saques_samples, pagamentos_samples)

([('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.833.642-**',
   '16167611395',
   'ABIGAIL DAGMAR MACHADO',
   '2020-01-24',
   253),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.332.032-**',
   '16120063855',
   'ADALGIZA NOGUEIRA DOS SANTOS',
   '2020-01-27',
   170),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.089.602-**',
   '21208225415',
   'ADEIDE RODRIGUES DAMASCENO',
   '2020-01-27',
   422),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.469.322-**',
   '16036929952',
   'ADELINA XUITES JACOB',
   '2020-01-21',
   457),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.217.922-**',
   '20500011774',
   'ADENILDA TAVARES DA SILVA',
   '2020-02-27',
   297),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
   '***.593.192-**',
   '20326549867',
   'ADENILZA MACHADO PINHEIRO',
   '2020-02-06',
   171),
  ('202001',
   '202001',
   'AC',
   643,
   'ACRELANDIA',
  